In [5]:
import pandas as pd
import json
import csv
import pickle
import gzip
import itertools
import csv

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

data_directory = '/Users/nolanthomas/Public/amazon/raw'
json_file = parse ( data_directory + '/metadata.json.gz' )

#findthese = ['Clothing, Shoes & Jewelry',
#  'Women']
#'Clothing']
findthese = {'Clothing, Shoes & Jewelry','Women'}
electronics = {'Electronics'}

#len(findthese.intersection(set(j['categories'][0])))

filtered = {}
meta_asins = {}
#i = 0
for j in json_file:
    #print (json.dumps(j, indent=4, sort_keys=True))
    asin = j['asin']
    meta_asins[asin] = 1
    #if asin in asins and 'categories' in j:
    if 'categories' in j:
        tags = set(itertools.chain.from_iterable(j['categories']))
        for c in j['categories']:
            if len(c) >= 4 and len(findthese.intersection(set(c))) == 2:
                if asin not in filtered:
                    filtered[asin] = {'asin':j['asin'],
                                   'category_tags':tags,
                                   'categories':j['categories']}
                if 'level4_women' not in filtered[asin]:
                        filtered[asin]['level4_women'] = c[3]
                elif c[3] < filtered[asin]['level4_women']:
                    filtered[asin]['level4_women'] = c[3]
            if len(c) >= 4 and len(electronics.intersection(set(c))) == 1:
                if asin not in filtered:
                    filtered[asin] = {'asin':j['asin'],
                                   'category_tags':tags,
                                   'categories':j['categories']}
                if 'level4_electronics' not in filtered[asin]:
                        filtered[asin]['level4_electronics'] = c[3]
                elif c[3] < filtered[asin]['level4_electronics']:
                    filtered[asin]['level4_electronics'] = c[3]

len(filtered)

with open( '/Users/nolanthomas/Public/amazon/interim/in_asin_cat_tags_level4.csv','w') as fh:
    csv_w = csv.writer(fh, delimiter='\t',
                       quotechar='"', quoting=csv.QUOTE_ALL) 
    csv_w.writerow(['asin','categories','category_tags','level4_women','level4_electronics'])
    for k, r in filtered.items():
        if 'level4_women' not in r:
            r['level4_women'] = 'Unknown'
        if 'level4_electronics' not in r:
            r['level4_electronics'] = 'Unknown'
        csv_w.writerow([r['asin'],r['categories'],r['category_tags'],r['level4_women'],r['level4_electronics']])
